## Tuning LightGBM Model

Dataset: 

- https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset?select=diabetes_binary_health_indicators_BRFSS2015.csv

**=========================================================**

### Data Preparation

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

#### Load Dataset

In [2]:
# I will use function to reduce Pandas dataframe memory usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
df = import_data('diabetes_binary_health_indicators_BRFSS2015(1).csv')

Memory usage of dataframe is 42.58 MB
Memory usage after optimization is: 10.64 MB
Decreased by 75.0%


In [4]:
pd.set_option('display.max_columns', None)
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


#### Variable Types

LightGBM is able to handle categorical variables out-of-the-box and I will use this feature.

However, in real life scenario, categorical features encoding might get better performance models.

In [5]:
x_columns = list(df.columns)
response_column = 'Diabetes_binary'
x_columns.remove(response_column)

categorical_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost',
       'DiffWalk', 'Sex',
       'Income']
numerical_columns = list(set(x_columns)-set(categorical_columns))
print("Response column:", response_column)
print("Categorical columns:", categorical_columns)
print("Numerical columns:", numerical_columns)

# This is only needed if you are planning to use Sklearn API
for column in categorical_columns:
    print("Processing column:", column)
    df[column] = df[column].astype('int64') # Convert float to integer then to categorical
    df[column] = df[column].astype('category')

Response column: Diabetes_binary
Categorical columns: ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex', 'Income']
Numerical columns: ['Education', 'Age', 'MentHlth', 'GenHlth', 'BMI', 'PhysHlth']
Processing column: HighBP
Processing column: HighChol
Processing column: CholCheck
Processing column: Smoker
Processing column: Stroke
Processing column: HeartDiseaseorAttack
Processing column: PhysActivity
Processing column: Fruits
Processing column: Veggies
Processing column: HvyAlcoholConsump
Processing column: AnyHealthcare
Processing column: NoDocbcCost
Processing column: DiffWalk
Processing column: Sex
Processing column: Income


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   Diabetes_binary       253680 non-null  float16 
 1   HighBP                253680 non-null  category
 2   HighChol              253680 non-null  category
 3   CholCheck             253680 non-null  category
 4   BMI                   253680 non-null  float16 
 5   Smoker                253680 non-null  category
 6   Stroke                253680 non-null  category
 7   HeartDiseaseorAttack  253680 non-null  category
 8   PhysActivity          253680 non-null  category
 9   Fruits                253680 non-null  category
 10  Veggies               253680 non-null  category
 11  HvyAlcoholConsump     253680 non-null  category
 12  AnyHealthcare         253680 non-null  category
 13  NoDocbcCost           253680 non-null  category
 14  GenHlth               253680 non-nul

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[x_columns], 
                                                    df[response_column], 
                                                    test_size=0.20, 
                                                    random_state=314, 
                                                    stratify=df[response_column])

In [8]:
print("Train dimensions:",X_train.shape)
print("Test dimensions:",X_test.shape)

Train dimensions: (202944, 21)
Test dimensions: (50736, 21)


In [9]:
df[response_column].value_counts()

Diabetes_binary
0.0    218334
1.0     35346
Name: count, dtype: int64

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202944 entries, 210000 to 68327
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   HighBP                202944 non-null  category
 1   HighChol              202944 non-null  category
 2   CholCheck             202944 non-null  category
 3   BMI                   202944 non-null  float16 
 4   Smoker                202944 non-null  category
 5   Stroke                202944 non-null  category
 6   HeartDiseaseorAttack  202944 non-null  category
 7   PhysActivity          202944 non-null  category
 8   Fruits                202944 non-null  category
 9   Veggies               202944 non-null  category
 10  HvyAlcoholConsump     202944 non-null  category
 11  AnyHealthcare         202944 non-null  category
 12  NoDocbcCost           202944 non-null  category
 13  GenHlth               202944 non-null  float16 
 14  MentHlth              202944 non-null

### Train LightGBM

In [11]:
X_train.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
210000,1,1,1,24.0,1,0,0,1,0,0,0,1,0,4.0,0.0,5.0,1,0,12.0,5.0,6
27201,0,1,1,30.0,0,0,0,1,1,1,0,1,0,2.0,2.0,1.0,0,0,6.0,6.0,7
31381,1,0,1,33.0,0,0,0,1,0,0,0,1,1,4.0,0.0,0.0,0,0,8.0,5.0,7
25244,1,0,1,22.0,0,0,0,1,1,1,0,1,0,2.0,30.0,0.0,1,1,10.0,5.0,5
32644,0,1,1,32.0,1,1,0,1,1,1,0,1,0,3.0,0.0,20.0,0,1,9.0,6.0,7


In [12]:
train_data = lgb.Dataset(data=X_train, label=y_train, params={"verbose":-1}, categorical_feature=categorical_columns)
test_data = lgb.Dataset(data=X_test, label=y_test, params={"verbose":-1}, categorical_feature=categorical_columns)
lgb_clf = lgb.train(params={"verbose":-1},
                    train_set=train_data)

In [13]:
from sklearn.metrics import roc_auc_score
print("AUC score on Test dataset:", roc_auc_score(y_test, lgb_clf.predict(X_test)))
print("AUC score on Train dataset:", roc_auc_score(y_train, lgb_clf.predict(X_train)))

AUC score on Test dataset: 0.8291245162663716
AUC score on Train dataset: 0.8391701749746283


#### Hyper-Parameters Optimization

First try produce pretty close AUC metric for both train and test dataset. I will look for ways to improve it.

In [14]:
from optuna.integration import LightGBMPruningCallback
import tqdm as notebook_tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import optuna
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
study_model_iteractions = {}

def objective(trial, X, y):
    param_grid = {
        "num_iterations": 10000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, step=0.05),
        "num_leaves": trial.suggest_int("num_leaves", 50, 150, step=5),
        "max_depth": trial.suggest_int("max_depth", 5, 20, step=2),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 1000, step=100),
        ##############################
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        #"lambda_l1": trial.suggest_float("lambda_l1", 0.01, 0.1, step=0.01),
        #"lambda_l2": trial.suggest_float("lambda_l2", 0.01, 0.1, step=0.01),
        ########################
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.8, 1.0, step=0.1),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [5]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.8, 1.0, step=0.1),
        #######################
        "is_unbalance": trial.suggest_categorical("is_unbalance",[True, False]),
        ########################
        "verbose": -1,
        "objective":"binary",
        "metric":"auc"
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    cv_iteractions = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[test_idx]

        train_data = lgb.Dataset(data=X_train, label=y_train, params={"verbose":-1}, categorical_feature=categorical_columns)
        valid_data = lgb.Dataset(data=X_valid, label=y_valid, params={"verbose":-1}, categorical_feature=categorical_columns)
        lgb_clf = lgb.train(params=param_grid,
                            train_set=train_data,
                            valid_sets=[valid_data],
                            categorical_feature=categorical_columns,
                            callbacks=[LightGBMPruningCallback(trial, "auc"),
                                        lgb.early_stopping(stopping_rounds=5)]  
                            )
        preds = lgb_clf.predict(X_valid)
        cv_scores[idx] = roc_auc_score(y_valid, preds)
        cv_iteractions[idx] = lgb_clf.best_iteration
    
    study_model_iteractions[trial.number] = np.mean(cv_iteractions)

    return np.mean(cv_scores)

In [15]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, n_trials=100)

[I 2024-12-09 17:28:18,633] A new study created in memory with name: LGBM Classifier
/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")


Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[45]	valid_0's auc: 0.83364
Training until validation scores don't improve for 5 rounds


/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")


Early stopping, best iteration is:
[51]	valid_0's auc: 0.825236
Training until validation scores don't improve for 5 rounds


/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")


...

/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")
[I 2024-12-09 17:29:54,551] Trial 94 pruned. Trial was pruned at iteration 0.
/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")
[I 2024-12-09 17:29:54,771] Trial 95 pruned. Trial was pruned at iteration 3.


Training until validation scores don't improve for 5 rounds


/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")
[I 2024-12-09 17:29:54,925] Trial 96 pruned. Trial was pruned at iteration 0.
/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")
[I 2024-12-09 17:29:55,093] Trial 97 pruned. Trial was pruned at iteration 1.
/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDe

Training until validation scores don't improve for 5 rounds


/Users/thienla/ba_venv/lib/python3.11/site-packages/lightgbm/engine.py:186: LGBMDeprecationWarning: Argument 'categorical_feature' to train() is deprecated and will be removed in a future release. Set 'categorical_feature' when calling lightgbm.Dataset() instead. See https://github.com/microsoft/LightGBM/issues/6435.
  _emit_dataset_kwarg_warning("train", "categorical_feature")
[I 2024-12-09 17:29:55,372] Trial 99 pruned. Trial was pruned at iteration 0.


In [16]:
print(f"\tBest value (AUC): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

print("Best model best_iteration:", study_model_iteractions[study.best_trial.number])

	Best value (AUC): 0.82992
	Best params:
		learning_rate: 0.11
		num_leaves: 60
		max_depth: 9
		min_data_in_leaf: 1000
		lambda_l1: 2.7746339916922e-05
		lambda_l2: 0.00020954551165614708
		bagging_fraction: 1.0
		bagging_freq: 5
		feature_fraction: 1.0
		is_unbalance: True
Best model best_iteration: 47.6


In [17]:
best_params = {"verbose": -1,
                "objective":"binary",
                "metric":"auc"
            }
for key,val in study.best_params.items():
    best_params[key] = val

best_params["num_iterations"] = int(study_model_iteractions[study.best_trial.number])
print(best_params)

{'verbose': -1, 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.11, 'num_leaves': 60, 'max_depth': 9, 'min_data_in_leaf': 1000, 'lambda_l1': 2.7746339916922e-05, 'lambda_l2': 0.00020954551165614708, 'bagging_fraction': 1.0, 'bagging_freq': 5, 'feature_fraction': 1.0, 'is_unbalance': True, 'num_iterations': 47}


#### Train LightGBM Best Parameters

In [18]:
train_data = lgb.Dataset(data=X_train, label=y_train, params={"verbose":-1}, categorical_feature=categorical_columns)
test_data = lgb.Dataset(data=X_test, label=y_test, params={"verbose":-1}, categorical_feature=categorical_columns)
best_lgb = lgb.train(params=best_params,
                    train_set=train_data) 

In [19]:
print("AUC score on Test dataset:", roc_auc_score(y_test, best_lgb.predict(X_test)))
print("AUC score on Train dataset:", roc_auc_score(y_train, best_lgb.predict(X_train)))

AUC score on Test dataset: 0.8296202804787243
AUC score on Train dataset: 0.8382715548697526


#### Summary

I got only very small improvement in AUC on Testing dataset after model tuning. However, even 1% improvement in the AUC just by tuning hyper-parameters is significant.